In [1]:
from sagemaker.feature_store.feature_group import FeatureGroup
from time import gmtime, strftime, sleep
from random import randint
import pandas as pd
import numpy as np
import subprocess
import sagemaker
import importlib
import logging
import time
import sys
import boto3
from datetime import datetime, timezone, date

In [2]:
logger = logging.getLogger('__name__')
logger.setLevel(logging.DEBUG)
logger.addHandler(logging.StreamHandler())

In [3]:
logger.info(f'Using SageMaker version: {sagemaker.__version__}')
logger.info(f'Using Boto3 version: {boto3.__version__}')

Using SageMaker version: 2.145.0
Using Boto3 version: 1.26.71


In [5]:
sagemaker_session = sagemaker.Session()
role = sagemaker.get_execution_role()
default_bucket = 'sm-fs-demo'
logger.info(f'Default S3 bucket = {default_bucket}')
prefix = 'sagemaker-feature-store'
region = sagemaker_session.boto_region_name

boto_session = boto3.Session(region_name=region)
sagemaker_client = boto_session.client(service_name='sagemaker', region_name=region)
featurestore_runtime = boto_session.client(service_name='sagemaker-featurestore-runtime', region_name=region)

Default S3 bucket = sm-fs-demo


In [6]:
def generate_event_timestamp():
    # naive datetime representing local time
    naive_dt = datetime.now()
    # take timezone into account
    aware_dt = naive_dt.astimezone()
    # time in UTC
    utc_dt = aware_dt.astimezone(timezone.utc)
    # transform to ISO-8601 format
    event_time = utc_dt.isoformat(timespec='milliseconds')
    event_time = event_time.replace('+00:00', 'Z')
    return event_time

In [7]:
feature_group_name ='cc-agg-fg'

In [8]:
print('describe fg', generate_event_timestamp())

ret = sagemaker_client.describe_feature_metadata(
    FeatureGroupName=feature_group_name,
    FeatureName="name" 
)

print('update fg', generate_event_timestamp(), '\n', ret['Parameters'])

sagemaker_client.update_feature_metadata(
    FeatureGroupName=feature_group_name,
    FeatureName="name",
    ParameterAdditions=[
        {"Key": "team", "Value": "mlops"},
        {"Key": "org1", "Value": "customer fin team"},
    ]
)

print('updated fg', generate_event_timestamp())

ret = sagemaker_client.describe_feature_metadata(
    FeatureGroupName=feature_group_name,
    FeatureName="name" 
)

print('describe modified fg', generate_event_timestamp(), '\n', ret['Parameters'])

describe fg 2023-04-10T18:46:34.202Z
update fg 2023-04-10T18:46:34.514Z 
 []
updated fg 2023-04-10T18:46:34.731Z
describe modified fg 2023-04-10T18:46:34.934Z 
 [{'Key': 'org1', 'Value': 'customer fin team'}, {'Key': 'team', 'Value': 'mlops'}]


In [9]:
# Search functions that returns features where either feature name, description or parameters (key/value pairs) match the search criteria
def search_features_using_string(search_string):
    response = sagemaker_client.search(
        Resource= "FeatureMetadata",
        SearchExpression={
            'Filters': [
                {
                    'Name': 'FeatureName',
                    'Operator': 'Contains',
                    'Value': search_string
                },
                {
                    'Name': 'Description',
                    'Operator': 'Contains',
                    'Value': search_string
                },
                {
                    'Name': 'AllParameters',
                    'Operator': 'Contains',
                    'Value': search_string
                }
            ],
            "Operator": "Or"
        },
    )
    # Displaying results in a DataFrame
    df=pd.json_normalize(response['Results'], max_level=1)
    df.columns = df.columns.map(lambda col: col.split(".")[1])
    df=df.drop('FeatureGroupArn', axis=1)
    return df

# Searching for Feature which contains "married" string in either feature name, description, or parameters
search_string="name"
search_features_using_string(search_string)

,FeatureGroupName,FeatureName,FeatureType,CreationTime,LastModifiedTime,Parameters
0,cc-agg-fg,name,String,2023-04-10 18:25:17+00:00,2023-04-10 18:46:34+00:00,"[{'Key': 'org1', 'Value': 'customer fin team'}..."
1,cc-agg-batch-fg,name,String,2023-04-10 16:46:24+00:00,2023-04-10 16:46:24+00:00,[]
2,cc-agg-chime-fg-v1,first_name,String,2023-04-08 23:02:56+00:00,2023-04-08 23:02:56+00:00,[]
3,cc-agg-chime-fg,name,String,2023-04-07 19:43:33+00:00,2023-04-08 21:46:35+00:00,"[{'Key': 'org', 'Value': 'customer fin team'},..."
4,cc-agg-update-test-1,name,String,2023-04-06 08:09:22+00:00,2023-04-06 08:09:22+00:00,[]
5,cc_agg_update_test,name,String,2023-04-06 07:41:34+00:00,2023-04-06 07:41:34+00:00,[]
6,cc-agg-batch-chime-fg,name,String,2023-04-06 06:49:17+00:00,2023-04-06 06:49:17+00:00,[]
